# Get tf-idf_SVD, euclidean-distance and cosine-distance matrices
The overall goal for this script is to iterate through **neuroscience-related R01s** for **each year**, get the abstracts for each of those projects and return three matrices and one dataframe:
1. **tf-idf_SVD**: This is a SVD-decomposed tf-idf wordcount matrix [n_abstracts, n_components]. I will use this for k-means clustering and affinity propagation (I think).
2. **cos_SVD**: This is a cosine distance matrix [n_abstracts, n_abstracts] generated from tf-idf_SVD. I will use this for Ward hierarchical clustering.
3. **euc_SVD**: This is an euclidean distance matrix [n_abstracts, n_abstracts] generated from tf-idf_SVD. I will also use this for Ward hierarchical clustering.
4. **df_neuro**: This is a dataframe, filtered for neuro-related R01s, containing information about each project.

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot

from string import punctuation
from nltk.corpus import stopwords
from nltk import PorterStemmer

%matplotlib inline

# Read abstracts and project csvs into dataframes

In [2]:
def get_abstracts_projects(year):
    """
    input: int -- year to analyze; [1985:2016] inclusive.
    returns: 2 dataframes -- project & abstract
    """
    abs_dir = './abstracts/'
    proj_dir = './projects/'
    
    project = pd.read_csv(proj_dir + 'RePORTER_PRJ_C_FY' + year + '.csv', encoding = "ISO-8859-1")
    abstract = pd.read_csv(abs_dir + 'RePORTER_PRJABS_C_FY' + year + '.csv',encoding = "ISO-8859-1")
    
    return project, abstract

# Clustering NIH data by common keywords

### Clean abstracts (stemming, lowercase, no punctuation, remove stopwords)

In [3]:
# get customized stopwords:
def customized_stopwords(to_append=[]):
    """
    to_append: list; what words do you want to exclude from your analysis, in addition to the standard 
    stopwords like 'the', 'and, 'of', and so on? See above to_append variable for examples.
    returns: list; stopwords including to_append list
    """
    stop = stopwords.words('english')
    stop = stop + to_append
    return stop

# get list of words that are lowercase, with punctuation removed and words stemmed.
def get_wordlist(abstract, stop = customized_stopwords()):
    """
    returns a list of lowercase words from abstract with punctuation and stopwords removed.
    """
    try:
        # make words lowercase
        words = abstract.lower()
        
        # take out all punctuation and split strings into a list of words
        words = (''.join(c for c in words if c not in punctuation)).split(' ')
        
        # remove stopwords
        words = [" ".join([w for w in word.split() if not w in stop]) for word in words]
        
        # stem words using Porter's Stemmer
        stemmed = []
        for word in words:
            try:
                word = PorterStemmer().stem(word)
            except IndexError:
                word = word
            if word != '' and word.isalpha():
                stemmed.append(word)
        words = stemmed

    except AttributeError:
        words = []
    return words

### Filter for neuro-related abstracts.
I defined a project to be neuroscience-related if the abstract mentioned "brain" or "neur*" at least twice every 100 non-stopwords.

In [17]:
def neuro_count(row):
    try:
        return (row.ABSTRACT_TEXT.count(' brain') + row.ABSTRACT_TEXT.count('neur'))
    except AttributeError:
        return 0
    
def wordlist_count(row):
    return len(row.wordlist)
def neuro_only(df, word_density=0.02):
    """
    input: dataframe
    word_density: how many neuro related words for every 100 words that are not stopwords in an abstract? Stopwords: the, and, or, not, etc.
    returns: dataframe containing neuro-related projects as defined above, with a column containing cleaned abstract keywords for analysis.
    """
    df['abs_neuro_count'] = df.apply(neuro_count, axis=1)
    df['wordlist'] = df.ABSTRACT_TEXT.apply(get_wordlist)
    df['wordlist_ct'] = df.apply(wordlist_count, axis=1)
    df['rel_neuro_count'] = df.abs_neuro_count / df.wordlist_ct
    
    # drop duplicates - I found that in some years, there are duplicate abstracts which really screwed up my hierarchical clustering
    df = df.drop_duplicates('ABSTRACT_TEXT')
    return df[df.rel_neuro_count >= word_density]

### Get feature vectors into a dataframe

In [5]:
def unique_words(df):
    """
    input: df containing 'wordlist' column
    returns: a dictionary containing unique words as keys, where dict[key][0] is total count of word over df
    """
    all_words = list(df.wordlist)
    all_words_list = [item for sublist in all_words for item in sublist]

    count_dict = {}
    
    # Get total word count for each word in dataset
    for word in all_words_list:
        if word not in count_dict.keys():
            count_dict[word] = [1]
        else:
            count_dict[word][0] += 1
        
    # for each project, get word count of each unique word
    for j in count_dict:
        count_dict[j] = count_dict[j] + ([0] * len(all_words))
    
    for i in range(len(all_words)):
        for word in all_words[i]:
            count_dict[word][i+1] += 1
    
    for key in count_dict:
        assert (count_dict[key][0] == sum(count_dict[key][1:])), print('this key,', key, 'is weird.')

    return count_dict

In [6]:
def fv_dict_to_df(df_neuro):
    """
    converts word_count_dict feature vectors into a dataframe with appropriate indices
    """
    word_count_dict = unique_words(df_neuro)
    
    feature_vec = pd.DataFrame(word_count_dict)
    fvec_index = ['Total'] + df_neuro.index.copy().tolist()
    feature_vec.index = fvec_index
    return feature_vec

### Compute tf-idf transformation vector and get td-idf transformation of feature vectors

In [7]:
def get_idf(feature_vector):
    """
    input: dataframe containing feature vectors where df.columns = words, and df.index is NIH funded project, and data contains count of word occurrences
    returns: numpy array for idf part
    """
    fv_no_tot = feature_vector.iloc[1:]
    fv_no_tot = fv_no_tot.replace(0, np.nan)
    
    # get idf = log (number of documents/number of documents with term t in it)
    return  np.log(len(feature_vector.iloc[0]) / (1 + np.array(fv_no_tot.count(axis=0))))
def get_tfidf(feature_vector, df_neuro):
    """
    feature_vector/total_words * idf
    """
    fv_no_tot = feature_vector.iloc[1:]
    
    # get tf
    total_words = df_neuro.ix[list(fv_no_tot.index), :].wordlist_ct
    tf = fv_no_tot.div(total_words, axis=0)
    
    # get idf
    idf = get_idf(feature_vector)
    
    # return td-idf
    return (tf * idf).dropna()
def clean_tfidf(tfidf):
    tfidf_del = ['studi', 'cell', 'brain', 'neuron', 'use', 'determin', 'effect', 'function', 'respons', 'specif', 
             'propos', 'activ', 'investig', 'examin', 'system', 'techniqu', 'chang', 'experi', 'understand', 'may',
            'aim']
    for i in tfidf_del:
        try:
            del tfidf[i]
        except:
            pass
    return tfidf

### Decompose tf-idf using SVD (latent semantic analysis)

In [8]:
def get_svd(tfidf):
    """
    apply SVD to tfidf
    """
    from sklearn.decomposition import TruncatedSVD
    
    svd = TruncatedSVD()
    svd.fit(tfidf)
    pyplot.plot(svd.explained_variance_ratio_.cumsum())

def recalc_svd(tfidf, components):
    """
    get user input for number of components that should explain >0.9 variance
    """
    
    from sklearn.decomposition import TruncatedSVD
    
    svd = TruncatedSVD(n_components = components)
    train_lsa = svd.fit_transform(tfidf)
    return train_lsa, svd

def plot_svd(svd, tfidf):
    """
    plots word contribution for top 10 SVD components, and variance explained
    """
    feat_names = list(tfidf.columns)

    for compNum in range(0, 10):
        comp = svd.components_[compNum]

        # Sort the weights in the first component, and get the indices
        indices = np.argsort(comp).tolist()

        # Reverse the indices, so we have the largest weights first.
        indices.reverse()

        # Grab the top 10 terms which have the highest weight in this component.        
        terms = [feat_names[weightIndex] for weightIndex in indices[0:10]]    
        weights = [comp[weightIndex] for weightIndex in indices[0:10]]    

        # Display these terms and their weights as a horizontal bar graph.    
        # The horizontal bar graph displays the first item on the bottom; reverse
        # the order of the terms so the biggest one is on top.
        terms.reverse()
        weights.reverse()
        positions = np.arange(10) + .5    # the bar centers on the y axis

        pyplot.figure(compNum)
        pyplot.barh(positions, weights, align='center')
        pyplot.yticks(positions, terms)
        pyplot.xlabel('Weight')
        pyplot.title('Strongest terms for component %d' % (compNum))
        pyplot.show()

### Get cosine similarity distance matrix

Cosine similarity is measured against the tf-idf matrix and can be used to generate a measure of similarity between each document and the other documents in the corpus (each synopsis among the synopses). Subtracting it from 1 provides cosine distance which I will use for plotting on a euclidean (2-dimensional) plane.

In [9]:
def get_cosine_matrix(tfidf_matrix):
    """
    input: tfidf matrix (n_samples X m_words)
    returns: distance matrix (n_samples X n_samples) (np ndarray)
    """
    from sklearn.metrics.pairwise import cosine_similarity
    return (1 - cosine_similarity(tfidf_matrix))

### Get Euclidean similarity distance matrix

In [10]:
def euc_dist_matrix(tfidf):
    from sklearn.metrics.pairwise import euclidean_distances
    return euclidean_distances(tfidf)

# Putting everything together

In [11]:
def get_matrices(year, grant_type='R01'):
    """
    input:
        year: string -- year to analyze; [1985:2016] inclusive.
        grant_type: string, what type of grant do you want to analyze? 
            Default = 'R01'
            Possibilities = ['D43', 'D71', 'DP1', 'DP2', 'DP3', 'DP5', 
            'DP7', 'E11', 'F30', 'F31', 'F32', 'F99', 'FI2', 'G08',
            'G11', 'G12', 'G13', 'G20', 'H25', 'H79', 'I01', 'I21',
            'I50', 'IK1', 'IK2', 'IP1', 'IS1', 'K01', 'K02', 'K05',
            'K06', 'K07', 'K08', 'K12', 'K18', 'K22', 'K23', 'K24',
            'K25', 'K26', 'K43', 'K76', 'K99', 'KL2', 'N01', 'N02',
            'N03', 'N43', 'N44', 'OT2', 'OT3', 'P01', 'P20', 'P2C',
            'P30', 'P40', 'P41', 'P42', 'P50', 'P51', 'P60', 'R00',
            'R01', 'R03', 'R13', 'R15', 'R18', 'R21', 'R24', 'R25',
            'R28', 'R33', 'R34', 'R35', 'R36', 'R37', 'R41', 'R42',
            'R43', 'R44', 'R49', 'R50', 'R56', 'R61', 'R90', 'RF1',
            'RL5', 'RM1', 'S10', 'S21', 'SB1', 'SC1', 'SC2', 'SC3',
            'T01', 'T03', 'T15', 'T32', 'T34', 'T35', 'T36', 'T37',
            'T42', 'T90', 'TL1', 'TL4', 'U01', 'U10', 'U13', 'U17',
            'U18', 'U19', 'U22', 'U24', 'U2C', 'U2G', 'U2R', 'U34',
            'U36', 'U38', 'U41', 'U42', 'U44', 'U45', 'U48', 'U50',
            'U51', 'U52', 'U54', 'U58', 'U59', 'U60', 'U61', 'U62',
            'U66', 'U88', 'U90', 'UC2', 'UC4', 'UC7', 'UE1', 'UF1',
            'UF2', 'UG1', 'UG3', 'UG4', 'UH2', 'UH3', 'UH4', 'UL1',
            'UM1', 'UM2', 'US4', 'UT2', 'Y01', 'Z01', 'ZIA', 'ZIB',
            'ZIC', 'ZID', 'ZIE', 'ZIF', 'ZIG', 'ZIH', 'ZII', 'ZIJ',
            'ZIK']
    returns: None
    saves: 
        fv_neuro_tfidf: dataframe
        cos_matrix: np nd array
        df_neuro_granttype: dataframe
        as csv files to 'matrices' directory
    """
    # get abstracts and projects for a single year
    project, abstract = get_abstracts_projects(year)
    
    # join abstracts to projects dataframe
    df = project.merge(abstract, on='APPLICATION_ID', how='left')
    
    # Implement clean abstracts and filter for neuro projects.
    df_neuro = neuro_only(df)
    
    # look only at R01s
    df_neuro_granttype = df_neuro[df_neuro.ACTIVITY == grant_type]
    
    # make word count feature vector
    fv_neuro = fv_dict_to_df(df_neuro_granttype)
    
    # get tf-idf matrix
    fv_neuro_tfidf = get_tfidf(fv_neuro, df_neuro)
    
    # clean tfidf
    fv_neuro_tfidf = clean_tfidf(fv_neuro_tfidf)
    
    # get tfidf SVD 
    print (get_svd(fv_neuro_tfidf))
    components = int(input('How many components to grab?: '))
    tfidf_svd, svd = recalc_svd(fv_neuro_tfidf, components)
    plot_svd(svd, fv_neuro_tfidf)
    
    # get cosine distance matrix
    cos_matrix = get_cosine_matrix(tfidf_svd)
    
    # get euclidean distance matrix
    euc_dist = euc_dist_matrix(tfidf_svd)
    
    # save files:
    # save neuro R01s dataframe
    df_neuro_granttype.to_csv('./matrices/neuro_df' + year + '_' + grant_type + '.csv')
    
    # save decomposed tfidf
    tfidf_svd.to_csv('./matrices/tfidfsvd_' + year + '_' + grant_type + '.csv')
    
    # save cosine distance matrix (using decomposed tfidf)
    np.savetxt('./matrices/cos_' + year + '_' + grant_type + '.csv', cos_matrix, delimiter = ',')
    
    # save euclidean distance matrix (using decomposed tfidf)
    np.savetxt('./matrices/cos_' + year + '_' + grant_type + '.csv', euc_dist, delimiter = ',')

# Implementation

In [ ]:
# make folder to contain matrices
try: 
    os.mkdir('./matrices/')
except FileExistsError:
    pass
        
# years = range(1985, 2017)
years = range(2005, 2006)
for year in years:
    get_matrices(str(year))
    print (str(year), 'data is done!')
print ('all done!')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2885: DtypeWarning: Columns (5,6,7,10,11,13,17,19,21,24,25,29,30,31,38) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
